In [3]:
import pandas as pd
import numpy as np
import random
from deap import base, creator, tools, algorithms

# User Input
teachers = input("Enter teachers (comma-separated): ").split(',')
subjects = input("Enter subjects (comma-separated): ").split(',')
time_slots = input("Enter time slots (comma-separated): ").split(',')
days = input("Enter days of the week (comma-separated): ").split(',')

# Genetic Algorithm Setup
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

def generate_individual():
    return [random.choice(teachers) for _ in range(len(days) * len(time_slots))]

def fitness_function(individual):
    # Simple fitness calculation (avoid consecutive same teacher slots)
    conflicts = sum(1 for i in range(len(individual)-1) if individual[i] == individual[i+1])
    return (conflicts,)

# Initialize Population
toolbox = base.Toolbox()
toolbox.register("individual", tools.initIterate, creator.Individual, generate_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", fitness_function)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)

# Running the GA
population = toolbox.population(n=50)
algorithms.eaSimple(population, toolbox, cxpb=0.5, mutpb=0.2, ngen=100, verbose=False)

# Extract Best Timetable
best = tools.selBest(population, k=1)[0]
schedule = np.array(best).reshape(len(days), len(time_slots))
df = pd.DataFrame(schedule, index=days, columns=time_slots)

# Display Timetable
display(df)


<class 'ModuleNotFoundError'>: No module named 'deap'

Error: One of the images could not be loaded.


In [9]:
import random
import numpy as np
import pandas as pd

# Define parameters
DAYS = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"]
TIME_SLOTS = ["9.00-9.55 AM", "9.55-10.50 AM", "11.10-12.05 PM", "12.05-1.00 PM", "2.00-2.55 PM", "2.55-3.50 PM", "3.50-4.45 PM"]
SEMESTERS = ["IV", "VI"]

# Faculty and Course Mapping (From User Instructions)
faculty_courses = {
    "Prof. Roopa G": ["BCS401", "BCSL404 (Lab)"],
    "Prof. Nagraj": ["BCS402", "BCSL606 (Lab)"],
    "Prof. Chaithanya": ["BCS403", "BCD685 (Lab - Alt)"],
    "Prof. Rohith": ["BAD601", "BXX405x (Lab)", "BCD685 (Lab - Alt)"],
    "Prof. Sapna": ["BXX613x", "BXX405x (Lab)", "BCD685 (Lab - Alt)"],
    "Prof. Akhila": ["BDS456x", "BXX657x (Lab)", "BCD685 (Lab - Alt)"],
    "Prof. Harshitha": ["BUHK408", "BIKS609", "BXX657x (Lab)", "BCSL404 (Lab)"],
    "Prof. SV Prasad": ["BDS602", "BCSL606 (Lab)"],
    "Prof. Sashi": ["BXX654x"],
}

# Lab Sessions (Fixed Durations)
lab_sessions = {
    "BCSL404 (Lab)": 2, "BCSL606 (Lab)": 2, "BXX657x (Lab)": 2,
    "BXX405x (Lab)": 2, "BCD685 (Lab)": 2
}

# Generate a random timetable (initial population)
def generate_timetable():
    timetable = {day: {slot: {} for slot in TIME_SLOTS} for day in DAYS}

    for faculty, courses in faculty_courses.items():
        assigned_slots = 0
        for course in courses:
            if assigned_slots >= 3:  # Max 3 slots per day per faculty
                continue

            day = random.choice(DAYS)
            slot = random.choice(TIME_SLOTS)

            # Check if slot is free
            if not timetable[day][slot]:
                timetable[day][slot] = {"Course": course, "Faculty": faculty}
                assigned_slots += 1

    return timetable

# Generate initial timetable
timetable = generate_timetable()

# Convert to DataFrame for better visualization
timetable_df = []
for day in DAYS:
    for slot in TIME_SLOTS:
        entry = timetable[day][slot]
        course = entry.get("Course", "Free Slot")
        faculty = entry.get("Faculty", "-")
        timetable_df.append([day, slot, course, faculty])

df_timetable = pd.DataFrame(timetable_df, columns=["Day", "Time Slot", "Course", "Faculty"])
df_timetable.head(50)  # Displaying first 20 rows for preview

,Day,Time Slot,Course,Faculty
0,Monday,9.00-9.55 AM,Free Slot,-
1,Monday,9.55-10.50 AM,Free Slot,-
2,Monday,11.10-12.05 PM,Free Slot,-
3,Monday,12.05-1.00 PM,BCSL404 (Lab),Prof. Roopa G
4,Monday,2.00-2.55 PM,Free Slot,-
5,Monday,2.55-3.50 PM,Free Slot,-
6,Monday,3.50-4.45 PM,BCS402,Prof. Nagraj
7,Tuesday,9.00-9.55 AM,BCD685 (Lab - Alt),Prof. Sapna
8,Tuesday,9.55-10.50 AM,Free Slot,-
9,Tuesday,11.10-12.05 PM,BCD685 (Lab - Alt),Prof. Rohith


In [3]:
import random
from sklearn.preprocessing import LabelEncoder
import math

class ProfSubMetaData:

    PSMAP = {}
    def addMetaData(self, nameID, subIDs, prefSlots):

        metaData = {"subIDs" : subIDs, "prefSlots" : prefSlots}
        self.PSMAP[nameID] = metaData

class EncodePhenotypes:
    sems = {}
    classrooms = {}
    profs = {}
    subs = {}
    days = {}
    slots = {}

    def __init__(self, sems, classrooms, profs, subs, days, slots):
        self.sems = self.encode(list(sems))
        self.classrooms = self.encode(list(classrooms))
        self.profs = self.encode(list(profs))
        self.subs = self.encode(list(subs))
        self.days = self.encode(list(days))
        self.slots = self.encode(list(slots))

    
    def encode(self, data):
        
        le = LabelEncoder()
        le.fit(data)
        temp = le.transform(data)
        EncodedData = {}
        for i in range(0, len(data)):
            bits = math.ceil(math.log(len(temp), 2))
            EncodedData[data[i]] = '0' * (bits-len(bin(i)[2:])) + bin(i)[2:]
        return EncodedData


faculty = {'VIR','LOH','AMO','GUI','NAY','CHT','KUT','AMI','SON','FER','DEV','BOR','PUR','GEE','PAL','MAN','ANK','SHR','JOH','JIL','SAM','MEH'}

S3Subs = {'MAT3', 'CAS', 'EDC', 'DSD', 'EMWT', 'EDCLAB', 'DSDLAB', 'TC', 'MATBC'}
S4Subs = {'SAS', 'MPI', 'LIC', 'TLA', 'SCT', 'MPILAB', 'LICLAB', 'EEM'}
S5Subs = {'ADC', 'DSP', 'PE1', 'PE2', 'CELAB', 'EMLAB', 'OE1', 'EE'}
S6Subs = {'CSE', 'VLSI', 'PE3', 'PE4', 'VLSILAB', 'ESDLAB', 'OE2', 'CLI'}
S7Subs = {'DC', 'PE5', 'DCLAB', 'OE3'}
S8Subs = {'ACE', 'PE6'}
AllSubs = S3Subs | S4Subs | S5Subs | S6Subs |S7Subs | S8Subs

classrooms = {'S1', 'S2', 'S3', 'S4', 'B1', 'B2', 'B3'}
semesters = {'S3', 'S4', 'S5', 'S6', 'S7', 'S8'}
days = {'mon', 'tue', 'wed', 'thu', 'fri'}
slots = {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20}


def makeRandomMetaData(MetaDataObj, minSubs=1, maxSubs=3, maxOverlap=0):
    subsLeft = AllSubs
    for f in faculty:
        r = random.choice(tuple(subsLeft))
        MetaDataObj.addMetaData(f, [r], prefSlots=random.randint(0, 6))
        subsLeft.remove(r)
    profsLeft = faculty
    for s in subsLeft:
        f = random.choice(tuple(profsLeft))
        MetaDataObj.PSMAP[f]['subIDs'].append(s)
        profsLeft.remove(f)

# assingn subjects to teachers randomly for test purpose
metadata = ProfSubMetaData()
makeRandomMetaData(metadata)

#this objects stores binary encoded dicionaries
x = EncodePhenotypes(sems=semesters, classrooms= classrooms, profs=faculty, subs=AllSubs, days=days, slots=slots)

print(x.subs)

class GenetiAlgo:
    '''
    ---This class is responsible for all the Genetic Operations and Manipulations---

    intialPopSize -> sets the inital population pool size
    numClasses -> the number different classes you want to make a tt for
    numSlots -> the total number of slots for a class per week
    numSlotBits -> the number of bits required to represent information about one slot
    '''

    initialPop = []
    initialPopSize = 100
    subs = {}
    classrooms = {}
    numClasses = 3
    numSlots = 20
    numSlotBits = None
    def __init__(self, subs, classrooms, initialPopSize=100, numClasses = 3, numSlots = 20):
        self.initialPopSize = initialPopSize
        self.subs = subs
        self.classrooms = classrooms
        self.initialPop = []
        self.numClasses = numClasses
        self.numSlots = numSlots
        self.numSlotBits = len(list(self.subs.values()[0]))+ len(list(self.classrooms.values()[0]))
    def generateIntialPop(self ):
        '''
        Randomly Populates intialPop with binary values from subjects and classrooms
        '''
        for i in range(0, self.initialPopSize):
            individual = ''
            # an individual solution will contain numClasses * numSlots amount of slots in total... i.e the consolidated time table
            for j in range(0, (self.numClasses * self.numSlots)):
                s = random.choice(list(self.subs.values()))
                c = random.choice(list(self.classrooms.values()))
                individual = individual + s + c
            self.initialPop.append(individual)
            

    def fitness(self, individual):
        '''
        This function calculates the feasibility or the goodness of an individual solutionS 
        ---Yet to be programmed---
        '''
        return 0

sol = GenetiAlgo(subs=x.subs, classrooms=x.classrooms)
sol.generateIntialPop()
#print(len(sol.initialPop[1]))

{'CSE': '00000', 'MPI': '00001', 'LICLAB': '00010', 'EMWT': '00011', 'EE': '00100', 'MATBC': '00101', 'MAT3': '00110', 'TLA': '00111', 'OE3': '01000', 'ESDLAB': '01001', 'DC': '01010', 'CAS': '01011', 'PE2': '01100', 'PE3': '01101', 'SAS': '01110', 'OE1': '01111', 'DSP': '10000'}


<class 'TypeError'>: 'dict_values' object is not subscriptable

In [5]:
# Re-import necessary packages after kernel reset
from collections import defaultdict

# Re-define time-related constants
DAYS = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"]
TIME_SLOTS = ["9:00-9:55", "9:55-10:50", "11:15-12:05", "12:10-1:00", "2:00-2:55", "2:55-3:50"]

# Reconstruct the course data
IV_SEM_COURSES = [
    "BCS401", "BCS403", "BAD601", "BUHK408", "BDS456x", "BXX613x", "BXX654x",
    "BIKS609", "BCS402", "BCSL404", "BXX405x", "BCD685", "BXX657x", "BCSL606"
]

VI_SEM_COURSES = [
    "BCS401", "BCS403", "BAD601", "BUHK408", "BDS456x", "BXX613x", "BXX654x",
    "BIKS609", "BCS402", "BCSL404", "BXX405x", "BCD685", "BXX657x", "BCSL606", "BDS602"
]

# Faculty assignment
faculty_courses = {
    "Prof. Nagraj": ["BCS402", "BCSL606"],
    "Prof. SV Prasad": ["BDS602", "BCSL606"],
    "Prof. Rohith": ["BAD601", "BCD685", "BXX405x"],
    "Prof. Sapna": ["BXX613x", "BCD685", "BXX405x"],
    "Prof. Sashi": ["BXX654x"],
    "Prof. Harshitha": ["BIKS609", "BXX657x", "BCSL404", "BUHK408"],
    "Prof. Akhila": ["BDS456x", "BCD685", "BXX657x"],
    "Prof. Chaithanya": ["BCS403", "BCD685"],
    "Prof. Roopa G": ["BCS401", "BCSL404"]
}

# Dummy Timetable placeholders for IV and VI (simplified to show structure)
timetable_IV = {
    day: {slot: None for slot in TIME_SLOTS} for day in DAYS
}
timetable_VI = {
    day: {slot: None for slot in TIME_SLOTS} for day in DAYS
}

# Sample filled slots for demo purposes
timetable_IV["Monday"]["9:00-9:55"] = "BCS401"
timetable_IV["Monday"]["9:55-10:50"] = "BCS403"
timetable_IV["Monday"]["11:15-12:10"] = "BCS402"
timetable_IV["Monday"]["12:10-1:00"] = "BDS456x"
timetable_IV["Monday"]["2:00-2:55"] = "BIKS609"
timetable_IV["Monday"]["2:55-3:50"] = "BUHK408"

timetable_VI["Tuesday"]["9:00-9:55"] = "BAD601"
timetable_VI["Tuesday"]["9:55-10:50"] = "BCS403"
timetable_VI["Tuesday"]["11:15-12:10"] = "BCS401"
timetable_VI["Tuesday"]["12:10-1:00"] = "BDS602"
timetable_VI["Tuesday"]["2:00-2:55"] = "BCSL606"
timetable_VI["Tuesday"]["2:55-3:50"] = "BCSL404"

# Combine timetables
combined_timetable = {
    day: {
        slot: timetable_IV[day][slot] if timetable_IV[day][slot] else timetable_VI[day][slot]
        for slot in TIME_SLOTS
    } for day in DAYS
}

# Organize timetable entries by semester
def organize_by_semester(timetable, courses):
    organized = defaultdict(lambda: {day: {slot: None for slot in TIME_SLOTS} for day in DAYS})
    for day in DAYS:
        for slot in TIME_SLOTS:
            subject = timetable[day][slot]
            if subject in courses["IV"]:
                organized["IV"][day][slot] = subject
            elif subject in courses["VI"]:
                organized["VI"][day][slot] = subject
    return organized

# Organize faculty timetable
def generate_faculty_timetable(timetable, faculty_courses):
    faculty_timetables = {faculty: {day: {slot: None for slot in TIME_SLOTS} for day in DAYS} for faculty in faculty_courses}
    
    for faculty, subjects in faculty_courses.items():
        for day in DAYS:
            for slot in TIME_SLOTS:
                subject = timetable[day][slot]
                if subject in subjects:
                    faculty_timetables[faculty][day][slot] = subject
    return faculty_timetables

# Display timetable in grid format
def display_timetable(timetable, title):
    print(f"\n📘 Timetable for {title} Semester:\n")
    print(f"{'Day':<12}{' | '.join(TIME_SLOTS)}")
    print("-" * 100)
    for day in DAYS:
        row = [day.ljust(12)]
        for slot in TIME_SLOTS:
            subject = timetable[day][slot]
            row.append(subject if subject else "Free")
        print(" | ".join(row))

# Display all faculty timetables
def display_faculty_timetables(faculty_timetables):
    for faculty, timetable in faculty_timetables.items():
        print(f"\n📘 Timetable for {faculty}:\n")
        print(f"{'Day':<12}{' | '.join(TIME_SLOTS)}")
        print("-" * 100)
        for day in DAYS:
            row = [day.ljust(12)]
            for slot in TIME_SLOTS:
                subject = timetable[day][slot]
                row.append(subject if subject else "Free")
            print(" | ".join(row))

# Process and display
semester_courses = {"IV": IV_SEM_COURSES, "VI": VI_SEM_COURSES}
sem_timetables = organize_by_semester(combined_timetable, semester_courses)
faculty_timetables = generate_faculty_timetable(combined_timetable, faculty_courses)

# Display outputs
for semester, timetable in sem_timetables.items():
    display_timetable(timetable, semester)

display_faculty_timetables(faculty_timetables)



📘 Timetable for IV Semester:

Day         9:00-9:55 | 9:55-10:50 | 11:15-12:10 | 12:10-1:00 | 2:00-2:55 | 2:55-3:50
----------------------------------------------------------------------------------------------------
Monday       | BCS401 | BCS403 | BCS402 | BDS456x | BIKS609 | BUHK408
Tuesday      | BAD601 | BCS403 | BCS401 | Free | BCSL606 | BCSL404
Wednesday    | Free | Free | Free | Free | Free | Free
Thursday     | Free | Free | Free | Free | Free | Free
Friday       | Free | Free | Free | Free | Free | Free
Saturday     | Free | Free | Free | Free | Free | Free

📘 Timetable for VI Semester:

Day         9:00-9:55 | 9:55-10:50 | 11:15-12:10 | 12:10-1:00 | 2:00-2:55 | 2:55-3:50
----------------------------------------------------------------------------------------------------
Monday       | Free | Free | Free | Free | Free | Free
Tuesday      | Free | Free | Free | BDS602 | Free | Free
Wednesday    | Free | Free | Free | Free | Free | Free
Thursday     | Free | Free | Free | Free

In [6]:
import random

# Define the time slots based on the given constraints
TIME_SLOTS = ["9:00-9:55 AM", "9:55-10:50 AM", "11:15-12:10 PM", "12:10-1:00 PM", "2:00-2:55 PM", "2:55-3:50 PM"]
DAYS = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"]

# Courses for IV and VI Semester
IV_SEM_COURSES = ["BCS401", "BCS402", "BCS403", "BDS456x", "BUHK408", "BXX405x (Lab)", "BCSL404 (Lab)"]
VI_SEM_COURSES = ["BAD601", "BDS602", "BXX613x", "BXX654x", "BIKS609", "BCD685 (Lab)", "BCSL606 (Lab)", "BXX657x (Lab)"]

# Lab courses (should be assigned to 2 continuous slots)
LAB_COURSES = {
    "BXX405x (Lab)", "BCSL404 (Lab)", "BCD685 (Lab)", "BCSL606 (Lab)", "BXX657x (Lab)"
}

# Faculty assignments (after modifications)
faculty_courses = {
    "Prof. Roopa G": ["BCS401", "BCSL404 (Lab)"],
    "Prof. Nagraj": ["BCS402", "BCSL606 (Lab)"],
    "Prof. Chaithanya": ["BCS403", "BCD685 (Lab)"],
    "Prof. Rohith": ["BAD601", "BXX405x (Lab)", "BCD685 (Lab)"],
    "Prof. Sapna": ["BXX613x", "BXX405x (Lab)", "BCD685 (Lab)"],
    "Prof. Akhila": ["BDS456x", "BXX657x (Lab)", "BCD685 (Lab)"],
    "Prof. Harshitha": ["BUHK408", "BIKS609", "BXX657x (Lab)", "BCSL404 (Lab)"],
    "Prof. SV Prasad": ["BDS602", "BCSL606 (Lab)"],
    "Prof. Sashi": ["BXX654x"],
}

# Generate initial random timetable
def generate_timetable():
    timetable = {day: {slot: None for slot in TIME_SLOTS} for day in DAYS}

    for course in IV_SEM_COURSES + VI_SEM_COURSES:
        assigned = False
        while not assigned:
            day = random.choice(DAYS)
            slot_index = random.randint(0, len(TIME_SLOTS) - 1)

            # Labs need 2 continuous slots
            if course in LAB_COURSES and slot_index < len(TIME_SLOTS) - 1:
                if not timetable[day][TIME_SLOTS[slot_index]] and not timetable[day][TIME_SLOTS[slot_index + 1]]:
                    timetable[day][TIME_SLOTS[slot_index]] = course
                    timetable[day][TIME_SLOTS[slot_index + 1]] = course
                    assigned = True
            # Regular classes
            elif course not in LAB_COURSES:
                if not timetable[day][TIME_SLOTS[slot_index]]:
                    timetable[day][TIME_SLOTS[slot_index]] = course
                    assigned = True

    return timetable

# Generate timetable for both semesters
timetable_IV = generate_timetable()
timetable_VI = generate_timetable()

# Convert timetable into a readable format
def display_timetable(timetable, semester):
    print(f"\nTimetable for {semester} Semester:\n")
    print(f"{'Day':<12}{' | '.join(TIME_SLOTS)}")
    print("-" * 100)

    for day in DAYS:
        row = [day.ljust(12)]
        for slot in TIME_SLOTS:
            row.append(timetable[day][slot] if timetable[day][slot] else "Free")
        print(" | ".join(row))

# Display both timetables
display_timetable(timetable_IV, "IV")
display_timetable(timetable_VI, "VI")




Timetable for IV Semester:

Day         9:00-9:55 AM | 9:55-10:50 AM | 11:15-12:10 PM | 12:10-1:00 PM | 2:00-2:55 PM | 2:55-3:50 PM
----------------------------------------------------------------------------------------------------
Monday       | Free | BXX657x (Lab) | BXX657x (Lab) | BXX405x (Lab) | BXX405x (Lab) | Free
Tuesday      | Free | Free | Free | Free | BXX654x | Free
Wednesday    | BCS401 | BDS456x | BCS403 | Free | Free | BCS402
Thursday     | BUHK408 | BDS602 | Free | BAD601 | BCSL404 (Lab) | BCSL404 (Lab)
Friday       | Free | BCD685 (Lab) | BCD685 (Lab) | BCSL606 (Lab) | BCSL606 (Lab) | Free
Saturday     | BXX613x | Free | BIKS609 | Free | Free | Free

Timetable for VI Semester:

Day         9:00-9:55 AM | 9:55-10:50 AM | 11:15-12:10 PM | 12:10-1:00 PM | 2:00-2:55 PM | 2:55-3:50 PM
----------------------------------------------------------------------------------------------------
Monday       | BCS401 | BAD601 | Free | BCD685 (Lab) | BCD685 (Lab) | Free
Tuesday      |

In [2]:
# Genetic Algorithm-based Timetable Generator for Multiple Semesters

import random
import copy
from collections import defaultdict
import pandas as pd

# Constants
DAYS = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"]
SLOTS_PER_DAY = 6
TOTAL_SLOTS = len(DAYS) * SLOTS_PER_DAY

BREAKS = [(1,)]  # Slot 2 is break (10:50 - 11:15)
LUNCH = [(3,)]   # Slot 4 is lunch (1:00 - 2:00)
LAB_BLOCK_SIZE = 2

# Example structure for courses per semester
# (Add more semesters dynamically as needed)
courses_per_semester = {
    'IV': [
        {'code': 'BCS401', 'type': 'Theory', 'faculty': 'Prof Roopa G', 'hours_per_week': 3},
        {'code': 'BCS402', 'type': 'Theory', 'faculty': 'Prof Nagraj', 'hours_per_week': 3},
        {'code': 'BCS403', 'type': 'Theory', 'faculty': 'Prof Chaithanya', 'hours_per_week': 3},
        {'code': 'BDS456x', 'type': 'Theory', 'faculty': 'Prof Akhila', 'hours_per_week': 3},
        {'code': 'BUHK408', 'type': 'Theory', 'faculty': 'Prof Harshitha', 'hours_per_week': 2},
        {'code': 'BIKS609', 'type': 'Theory', 'faculty': 'Prof Harshitha', 'hours_per_week': 3},
        {'code': 'BXX405x', 'type': 'Lab', 'faculty': ['Prof Rohith', 'Prof Sapna'], 'hours_per_week': 2},
        {'code': 'BCSL404', 'type': 'Lab', 'faculty': ['Prof Roopa G', 'Prof Harshitha'], 'hours_per_week': 4},
    ],
    'VI': [
        {'code': 'BAD601', 'type': 'Theory', 'faculty': 'Prof Rohith', 'hours_per_week': 3},
        {'code': 'BDS602', 'type': 'Theory', 'faculty': 'Prof SV Prasad', 'hours_per_week': 3},
        {'code': 'BXX613x', 'type': 'Theory', 'faculty': 'Prof Sapna', 'hours_per_week': 3},
        {'code': 'BXX654x', 'type': 'Theory', 'faculty': 'Prof Sashi', 'hours_per_week': 3},
        {'code': 'BIKS609', 'type': 'Theory', 'faculty': 'Prof Harshitha', 'hours_per_week': 3},
        {'code': 'BCD685', 'type': 'Lab', 'faculty': ['Prof Rohith', 'Prof Sapna', 'Prof Akhila', 'Prof Chaithanya'], 'hours_per_week': 4},
        {'code': 'BCSL606', 'type': 'Lab', 'faculty': ['Prof SV Prasad', 'Prof Nagraj'], 'hours_per_week': 2},
        {'code': 'BXX657x', 'type': 'Lab', 'faculty': ['Prof Akhila', 'Prof Harshitha'], 'hours_per_week': 2},
    ],
}

# Helper function to generate empty timetable
def generate_empty_timetable():
    return [[None for _ in range(SLOTS_PER_DAY)] for _ in range(len(DAYS))]

# Genetic Algorithm
class TimetableGA:
    def __init__(self, courses_by_semester, population_size=30, generations=100):
        self.courses_by_semester = courses_by_semester
        self.population_size = population_size
        self.generations = generations
        self.population = []

    def initialize_population(self):
        self.population = []
        for _ in range(self.population_size):
            chromosome = {}
            for sem, courses in self.courses_by_semester.items():
                timetable = generate_empty_timetable()
                slots = list((day, slot) for day in range(len(DAYS)) for slot in range(SLOTS_PER_DAY))
                random.shuffle(slots)
                
                for course in courses:
                    hrs = course['hours_per_week']
                    placed = 0
                    if course['type'] == 'Lab':
                        while placed < hrs:
                            for d in range(len(DAYS)):
                                for s in range(SLOTS_PER_DAY - 1):
                                    if timetable[d][s] is None and timetable[d][s+1] is None and (s, s+1) not in BREAKS + LUNCH:
                                        timetable[d][s] = course['code']
                                        timetable[d][s+1] = course['code']
                                        placed += 2
                                        break
                                if placed >= hrs:
                                    break
                    else:
                        for d, s in slots:
                            if timetable[d][s] is None and (s,) not in BREAKS + LUNCH:
                                timetable[d][s] = course['code']
                                placed += 1
                                if placed == hrs:
                                    break
                chromosome[sem] = timetable
            self.population.append(chromosome)

    def fitness(self, chromosome):
        score = 0
        faculty_schedule = defaultdict(list)

        for sem, timetable in chromosome.items():
            courses = {c['code']: c for c in self.courses_by_semester[sem]}
            for d in range(len(DAYS)):
                for s in range(SLOTS_PER_DAY):
                    subject = timetable[d][s]
                    if subject:
                        course = courses.get(subject)
                        if course:
                            if course['type'] == 'Lab' and s < SLOTS_PER_DAY - 1:
                                if timetable[d][s+1] != subject:
                                    score -= 1  # Lab not continuous
                            if isinstance(course['faculty'], list):
                                for f in course['faculty']:
                                    faculty_schedule[f].append((d, s))
                            else:
                                faculty_schedule[course['faculty']].append((d, s))
        # Check for faculty conflicts
        for faculty, slots in faculty_schedule.items():
            seen = set()
            for slot in slots:
                if slot in seen:
                    score -= 1  # Faculty conflict
                seen.add(slot)
        return score

    def crossover(self, parent1, parent2):
        crossover_point = random.randint(0, len(self.courses_by_semester)-1)
        semesters = list(self.courses_by_semester.keys())
        child = {}
        for i, sem in enumerate(semesters):
            child[sem] = copy.deepcopy(parent1[sem] if i <= crossover_point else parent2[sem])
        return child

    def mutate(self, chromosome):
        # Randomly swap a couple of entries in one timetable
        sem = random.choice(list(chromosome.keys()))
        timetable = chromosome[sem]
        d1, d2 = random.randint(0, 5), random.randint(0, 5)
        s1, s2 = random.randint(0, 5), random.randint(0, 5)
        timetable[d1][s1], timetable[d2][s2] = timetable[d2][s2], timetable[d1][s1]

    def run(self):
        self.initialize_population()
        for _ in range(self.generations):
            self.population.sort(key=self.fitness, reverse=True)
            next_gen = self.population[:5]  # Elitism
            while len(next_gen) < self.population_size:
                parent1, parent2 = random.sample(self.population[:10], 2)
                child = self.crossover(parent1, parent2)
                if random.random() < 0.2:
                    self.mutate(child)
                next_gen.append(child)
            self.population = next_gen
        return self.population[0]  # Best timetable

# Run GA
ga = TimetableGA(courses_per_semester)
best_timetable = ga.run()

# Display
for sem, table in best_timetable.items():
    df = pd.DataFrame(table, index=DAYS, columns=[f"Slot {i+1}" for i in range(SLOTS_PER_DAY)])
    print(f"\n\n=== Timetable for {sem} Semester ===")
    print(df)




=== Timetable for IV Semester ===
            Slot 1   Slot 2   Slot 3  Slot 4   Slot 5   Slot 6
Monday      BCS401     None   BCS402  BCS401  BXX405x  BXX405x
Tuesday     BCS402  BCSL404  BCSL404    None     None     None
Wednesday  BUHK408     None  BUHK408    None  BDS456x     None
Thursday   BCSL404  BCSL404  BDS456x    None   BCS403   BCS401
Friday     BIKS609     None   BCS402    None   BCS403  BDS456x
Saturday    BCS403     None     None    None  BIKS609  BIKS609


=== Timetable for VI Semester ===
            Slot 1  Slot 2   Slot 3   Slot 4   Slot 5   Slot 6
Monday     BXX613x    None  BIKS609     None   BDS602     None
Tuesday     BCD685  BCD685  BCSL606  BCSL606  BXX657x  BXX657x
Wednesday   BCD685  BCD685     None     None  BXX613x  BXX654x
Thursday   BXX613x    None  BIKS609     None   BAD601     None
Friday      BDS602    None   BDS602     None   BAD601     None
Saturday   BXX654x    None  BIKS609     None  BXX654x   BAD601


In [1]:
# Genetic Algorithm-based Timetable Generator for Multiple Semesters

import random
import copy
from collections import defaultdict
import pandas as pd

# Constants
DAYS = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"]
SLOTS_PER_DAY = 6
TOTAL_SLOTS = len(DAYS) * SLOTS_PER_DAY

BREAKS = [(1,)]  # Slot 2 is break (10:50 - 11:15)
LUNCH = [(3,)]   # Slot 4 is lunch (1:00 - 2:00)
LAB_BLOCK_SIZE = 2

# Example structure for courses per semester
# (Add more semesters dynamically as needed)
courses_per_semester = {
    'IV': [
        {'code': 'BCS401', 'type': 'Theory', 'faculty': 'Prof Roopa G', 'hours_per_week': 3},
        {'code': 'BCS402', 'type': 'Theory', 'faculty': 'Prof Nagraj', 'hours_per_week': 3},
        {'code': 'BCS403', 'type': 'Theory', 'faculty': 'Prof Chaithanya', 'hours_per_week': 3, 'priority': True, 'preferred_day': 'Monday', 'preferred_slot': 1},
        {'code': 'BDS456x', 'type': 'Theory', 'faculty': 'Prof Akhila', 'hours_per_week': 3},
        {'code': 'BUHK408', 'type': 'Theory', 'faculty': 'Prof Harshitha', 'hours_per_week': 2},
        {'code': 'BIKS609', 'type': 'Theory', 'faculty': 'Prof Harshitha', 'hours_per_week': 3},
        {'code': 'BXX405x', 'type': 'Lab', 'faculty': ['Prof Rohith', 'Prof Sapna'], 'hours_per_week': 2, 'priority': True, 'preferred_day': 'Wednesday', 'preferred_slot': 1},
        {'code': 'BCSL404', 'type': 'Lab', 'faculty': ['Prof Roopa G', 'Prof Harshitha'], 'hours_per_week': 4},
    ],
    'VI': [
        {'code': 'BAD601', 'type': 'Theory', 'faculty': 'Prof Rohith', 'hours_per_week': 3},
        {'code': 'BDS602', 'type': 'Theory', 'faculty': 'Prof SV Prasad', 'hours_per_week': 3},
        {'code': 'BXX613x', 'type': 'Theory', 'faculty': 'Prof Sapna', 'hours_per_week': 3},
        {'code': 'BXX654x', 'type': 'Theory', 'faculty': 'Prof Sashi', 'hours_per_week': 3},
        {'code': 'BIKS609', 'type': 'Theory', 'faculty': 'Prof Harshitha', 'hours_per_week': 3},
        {'code': 'BCD685', 'type': 'Lab', 'faculty': ['Prof Rohith', 'Prof Sapna', 'Prof Akhila', 'Prof Chaithanya'], 'hours_per_week': 4},
        {'code': 'BCSL606', 'type': 'Lab', 'faculty': ['Prof SV Prasad', 'Prof Nagraj'], 'hours_per_week': 2},
        {'code': 'BXX657x', 'type': 'Lab', 'faculty': ['Prof Akhila', 'Prof Harshitha'], 'hours_per_week': 2,'priority': True, 'preferred_day': 'Wednesday', 'preferred_slot': 1},
    ],
}

# Helper function to generate empty timetable
def generate_empty_timetable():
    return [[None for _ in range(SLOTS_PER_DAY)] for _ in range(len(DAYS))]

# Genetic Algorithm
class TimetableGA:
    def __init__(self, courses_by_semester, population_size=30, generations=100):
        self.courses_by_semester = courses_by_semester
        self.population_size = population_size
        self.generations = generations
        self.population = []

    def initialize_population(self):
        self.population = []
        for _ in range(self.population_size):
            chromosome = {}
            for sem, courses in self.courses_by_semester.items():
                timetable = generate_empty_timetable()
                slots = list((day, slot) for day in range(len(DAYS)) for slot in range(SLOTS_PER_DAY))
                random.shuffle(slots)

                for course in courses:
                    hrs = course['hours_per_week']
                    placed = 0
                    if course['type'] == 'Lab':
                        while placed < hrs:
                            for d in range(len(DAYS)):
                                for s in range(SLOTS_PER_DAY - 1):
                                    if timetable[d][s] is None and timetable[d][s+1] is None and (s, s+1) not in BREAKS + LUNCH:
                                        timetable[d][s] = course['code']
                                        timetable[d][s+1] = course['code']
                                        placed += 2
                                        break
                                if placed >= hrs:
                                    break
                    else:
                        for d, s in slots:
                            if timetable[d][s] is None and (s,) not in BREAKS + LUNCH:
                                timetable[d][s] = course['code']
                                placed += 1
                                if placed == hrs:
                                    break
                chromosome[sem] = timetable
            self.population.append(chromosome)

    def fitness(self, chromosome):
        score = 0
        faculty_schedule = defaultdict(list)

        for sem, timetable in chromosome.items():
            courses = {c['code']: c for c in self.courses_by_semester[sem]}
            for d in range(len(DAYS)):
                for s in range(SLOTS_PER_DAY):
                    subject = timetable[d][s]
                    if subject:
                        course = courses.get(subject)
                        if course:
                            if course['type'] == 'Lab' and s < SLOTS_PER_DAY - 1:
                                if timetable[d][s+1] != subject:
                                    score -= 1  # Lab not continuous
                            if isinstance(course['faculty'], list):
                                for f in course['faculty']:
                                    faculty_schedule[f].append((d, s))
                            else:
                                faculty_schedule[course['faculty']].append((d, s))

                            # Priority handling
                            if course.get('priority'):
                                if course.get('preferred_day') and DAYS[d] == course['preferred_day']:
                                    score += 2
                                if course.get('preferred_slot') == s:
                                    score += 2

        # Check for faculty conflicts
        for faculty, slots in faculty_schedule.items():
            seen = set()
            for slot in slots:
                if slot in seen:
                    score -= 1  # Faculty conflict
                seen.add(slot)
        return score

    def crossover(self, parent1, parent2):
        crossover_point = random.randint(0, len(self.courses_by_semester)-1)
        semesters = list(self.courses_by_semester.keys())
        child = {}
        for i, sem in enumerate(semesters):
            child[sem] = copy.deepcopy(parent1[sem] if i <= crossover_point else parent2[sem])
        return child

    def mutate(self, chromosome):
        # Randomly swap a couple of entries in one timetable
        sem = random.choice(list(chromosome.keys()))
        timetable = chromosome[sem]
        d1, d2 = random.randint(0, 5), random.randint(0, 5)
        s1, s2 = random.randint(0, 5), random.randint(0, 5)
        timetable[d1][s1], timetable[d2][s2] = timetable[d2][s2], timetable[d1][s1]

    def run(self):
        self.initialize_population()
        for _ in range(self.generations):
            self.population.sort(key=self.fitness, reverse=True)
            next_gen = self.population[:5]  # Elitism
            while len(next_gen) < self.population_size:
                parent1, parent2 = random.sample(self.population[:10], 2)
                child = self.crossover(parent1, parent2)
                if random.random() < 0.2:
                    self.mutate(child)
                next_gen.append(child)
            self.population = next_gen
        return self.population[0]  # Best timetable

# Run GA
ga = TimetableGA(courses_per_semester)
best_timetable = ga.run()

# Display
for sem, table in best_timetable.items():
    df = pd.DataFrame(table, index=DAYS, columns=[f"Slot {i+1}" for i in range(SLOTS_PER_DAY)])
    print(f"\n\n=== Timetable for {sem} Semester ===")
    print(df)




=== Timetable for IV Semester ===
            Slot 1   Slot 2   Slot 3 Slot 4   Slot 5   Slot 6
Monday     BIKS609   BCS403   BCS403   None   BCS401  BCSL404
Tuesday    BUHK408  BXX405x  BXX405x   None  BIKS609   BCS402
Wednesday   BCS401     None   BCS402   None   BCS402     None
Thursday   BDS456x   BCS403   BCS401   None     None  BIKS609
Friday     BCSL404  BCSL404  BCSL404   None  BDS456x     None
Saturday   BDS456x     None     None   None  BUHK408     None


=== Timetable for VI Semester ===
            Slot 1   Slot 2   Slot 3   Slot 4   Slot 5   Slot 6
Monday      BDS602     None  BXX613x   BCD685   BCD685   BCD685
Tuesday    BXX613x     None  BCSL606  BCSL606   BDS602  BIKS609
Wednesday   BAD601  BXX657x  BXX657x  BIKS609     None   BCD685
Thursday    BDS602     None  BXX654x     None  BXX654x     None
Friday        None     None     None     None  BXX654x  BIKS609
Saturday    BAD601     None     None     None   BAD601  BXX613x


In [1]:
# Genetic Algorithm-based Timetable Generator for Multiple Semesters

import random
import copy
from collections import defaultdict
import pandas as pd

# Constants
DAYS = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"]
SLOTS_PER_DAY = 6
TOTAL_SLOTS = len(DAYS) * SLOTS_PER_DAY

BREAKS = [(1,)]  # Slot 2 is break (10:50 - 11:15)
LUNCH = [(3,)]   # Slot 4 is lunch (1:00 - 2:00)
LAB_BLOCK_SIZE = 2

# Example structure for courses per semester
# (Add more semesters dynamically as needed)
courses_per_semester = {
    'IV': [
        {'code': 'BCS401', 'type': 'Theory', 'faculty': 'Prof Roopa G', 'hours_per_week': 3},
        {'code': 'BCS402', 'type': 'Theory', 'faculty': 'Prof Nagraj', 'hours_per_week': 3},
        {'code': 'BCS403', 'type': 'Theory', 'faculty': 'Prof Chaithanya', 'hours_per_week': 3, 'priority': True, 'preferred_day': 'Monday', 'preferred_slot': 1},
        {'code': 'BDS456x', 'type': 'Theory', 'faculty': 'Prof Akhila', 'hours_per_week': 3},
        {'code': 'BUHK408', 'type': 'Theory', 'faculty': 'Prof Harshitha', 'hours_per_week': 2},
        {'code': 'BIKS609', 'type': 'Theory', 'faculty': 'Prof Harshitha', 'hours_per_week': 3},
        {'code': 'BXX405x', 'type': 'Lab', 'faculty': ['Prof Rohith', 'Prof Sapna'], 'hours_per_week': 2, 'priority': True, 'preferred_day': 'Wednesday', 'preferred_slot': 1},
        {'code': 'BCSL404', 'type': 'Lab', 'faculty': ['Prof Roopa G', 'Prof Harshitha'], 'hours_per_week': 4},
    ],
    'VI': [
        {'code': 'BAD601', 'type': 'Theory', 'faculty': 'Prof Rohith', 'hours_per_week': 3},
        {'code': 'BDS602', 'type': 'Theory', 'faculty': 'Prof SV Prasad', 'hours_per_week': 3},
        {'code': 'BXX613x', 'type': 'Theory', 'faculty': 'Prof Sapna', 'hours_per_week': 3},
        {'code': 'BXX654x', 'type': 'Theory', 'faculty': 'Prof Sashi', 'hours_per_week': 3},
        {'code': 'BIKS609', 'type': 'Theory', 'faculty': 'Prof Harshitha', 'hours_per_week': 3},
        {'code': 'BCD685', 'type': 'Lab', 'faculty': ['Prof Rohith', 'Prof Sapna', 'Prof Akhila', 'Prof Chaithanya'], 'hours_per_week': 4},
        {'code': 'BCSL606', 'type': 'Lab', 'faculty': ['Prof SV Prasad', 'Prof Nagraj'], 'hours_per_week': 2},
        {'code': 'BXX657x', 'type': 'Lab', 'faculty': ['Prof Akhila', 'Prof Harshitha'], 'hours_per_week': 2,'priority': True, 'preferred_day': 'Wednesday', 'preferred_slot': 1},
    ],
}

# Helper function to generate empty timetable
def generate_empty_timetable():
    return [[None for _ in range(SLOTS_PER_DAY)] for _ in range(len(DAYS))]

# Genetic Algorithm
class TimetableGA:
    def __init__(self, courses_by_semester, population_size=30, generations=100):
        self.courses_by_semester = courses_by_semester
        self.population_size = population_size
        self.generations = generations
        self.population = []

    def initialize_population(self):
        self.population = []
        for _ in range(self.population_size):
            chromosome = {}
            for sem, courses in self.courses_by_semester.items():
                timetable = generate_empty_timetable()
                slots = list((day, slot) for day in range(len(DAYS)) for slot in range(SLOTS_PER_DAY))
                random.shuffle(slots)

                for course in courses:
                    hrs = course['hours_per_week']
                    placed = 0
                    if course['type'] == 'Lab':
                        while placed < hrs:
                            for d in range(len(DAYS)):
                                for s in range(SLOTS_PER_DAY - 1):
                                    if timetable[d][s] is None and timetable[d][s+1] is None and (s, s+1) not in BREAKS + LUNCH:
                                        timetable[d][s] = course['code']
                                        timetable[d][s+1] = course['code']
                                        placed += 2
                                        break
                                if placed >= hrs:
                                    break
                    else:
                        for d, s in slots:
                            if timetable[d][s] is None and (s,) not in BREAKS + LUNCH:
                                timetable[d][s] = course['code']
                                placed += 1
                                if placed == hrs:
                                    break
                chromosome[sem] = timetable
            self.population.append(chromosome)

    def fitness(self, chromosome):
        score = 0
        faculty_schedule = defaultdict(list)

        for sem, timetable in chromosome.items():
            courses = {c['code']: c for c in self.courses_by_semester[sem]}
            for d in range(len(DAYS)):
                for s in range(SLOTS_PER_DAY):
                    subject = timetable[d][s]
                    if subject is not None and subject in courses:
                        course = courses[subject]
                        if course['type'] == 'Lab' and s < SLOTS_PER_DAY - 1:
                            if timetable[d][s+1] != subject:
                                score -= 1  # Lab not continuous
                                if isinstance(course['faculty'], list):
                                    for f in course['faculty']:
                                        faculty_schedule[f].append((d, s))
                                    else:
                                        faculty_schedule[course['faculty']].append((d, s))


                            # Priority handling
                            if course.get('priority'):
                                if course.get('preferred_day') and DAYS[d] == course['preferred_day']:
                                    score += 2
                                if course.get('preferred_slot') == s:
                                    score += 2

        # Check for faculty conflicts
        for faculty, slots in faculty_schedule.items():
            seen = set()
            for slot in slots:
                if slot in seen:
                    score -= 1  # Faculty conflict
                seen.add(slot)
        return score

    def crossover(self, parent1, parent2):
        crossover_point = random.randint(0, len(self.courses_by_semester)-1)
        semesters = list(self.courses_by_semester.keys())
        child = {}
        for i, sem in enumerate(semesters):
            child[sem] = copy.deepcopy(parent1[sem] if i <= crossover_point else parent2[sem])
        return child

    def mutate(self, chromosome):
        # Randomly swap a couple of entries in one timetable
        sem = random.choice(list(chromosome.keys()))
        timetable = chromosome[sem]
        d1, d2 = random.randint(0, 5), random.randint(0, 5)
        s1, s2 = random.randint(0, 5), random.randint(0, 5)
        timetable[d1][s1], timetable[d2][s2] = timetable[d2][s2], timetable[d1][s1]

    def run(self):
        self.initialize_population()
        for _ in range(self.generations):
            self.population.sort(key=self.fitness, reverse=True)
            next_gen = self.population[:5]  # Elitism
            while len(next_gen) < self.population_size:
                parent1, parent2 = random.sample(self.population[:10], 2)
                child = self.crossover(parent1, parent2)
                if random.random() < 0.2:
                    self.mutate(child)
                next_gen.append(child)
            self.population = next_gen
        return self.population[0]  # Best timetable

# Run GA
ga = TimetableGA(courses_per_semester)
best_timetable = ga.run()

# Display
for sem, table in best_timetable.items():
    df = pd.DataFrame(table, index=DAYS, columns=[f"Slot {i+1}" for i in range(SLOTS_PER_DAY)])
    print(f"\n\n=== Timetable for {sem} Semester ===")
    print(df)




<class 'TypeError'>: unhashable type: 'list'

In [9]:
import pandas as pd
import matplotlib 

In [ ]:
# Genetic Algorithm-based Timetable Generator for Multiple Semesters

import random
import copy
from collections import defaultdict
import pandas as pd

# Constants
DAYS = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"]
SLOTS_PER_DAY = 6
TOTAL_SLOTS = len(DAYS) * SLOTS_PER_DAY

BREAKS = [(1,)]  # Slot 2 is break (10:50 - 11:15)
LUNCH = [(3,)]   # Slot 4 is lunch (1:00 - 2:00)
LAB_BLOCK_SIZE = 2

# Example structure for courses per semester
# (Add more semesters dynamically as needed)
courses_per_semester = {
    'IV': [
        {'code': 'BCS401', 'type': 'Theory', 'faculty': 'Prof Roopa G', 'hours_per_week': 3},
        {'code': 'BCS402', 'type': 'Theory', 'faculty': 'Prof Nagraj', 'hours_per_week': 3},
        {'code': 'BCS403', 'type': 'Theory', 'faculty': 'Prof Chaithanya', 'hours_per_week': 3},
        {'code': 'BDS456x', 'type': 'Theory', 'faculty': 'Prof Akhila', 'hours_per_week': 3},
        {'code': 'BUHK408', 'type': 'Theory', 'faculty': 'Prof Harshitha', 'hours_per_week': 2},
        {'code': 'BIKS609', 'type': 'Theory', 'faculty': 'Prof Harshitha', 'hours_per_week': 3},
        {'code': 'BXX405x', 'type': 'Lab', 'faculty': ['Prof Rohith', 'Prof Sapna'], 'hours_per_week': 2},
        {'code': 'BCSL404', 'type': 'Lab', 'faculty': ['Prof Roopa G', 'Prof Harshitha'], 'hours_per_week': 4, 'priority': True, 'preferred_day': 'Wednesday', 'preferred_slot': 1},
    ],
    'VI': [
        {'code': 'BAD601', 'type': 'Theory', 'faculty': 'Prof Rohith', 'hours_per_week': 3},
        {'code': 'BDS602', 'type': 'Theory', 'faculty': 'Prof SV Prasad', 'hours_per_week': 3},
        {'code': 'BXX613x', 'type': 'Theory', 'faculty': 'Prof Sapna', 'hours_per_week': 3},
        {'code': 'BXX654x', 'type': 'Theory', 'faculty': 'Prof Sashi', 'hours_per_week': 3},
        {'code': 'BIKS609', 'type': 'Theory', 'faculty': 'Prof Harshitha', 'hours_per_week': 3},
        {'code': 'BCD685', 'type': 'Lab', 'faculty': ['Prof Rohith', 'Prof Sapna', 'Prof Akhila', 'Prof Chaithanya'], 'hours_per_week': 4},
        {'code': 'BCSL606', 'type': 'Lab', 'faculty': ['Prof SV Prasad', 'Prof Nagraj'], 'hours_per_week': 2},
        {'code': 'BXX657x', 'type': 'Lab', 'faculty': ['Prof Akhila', 'Prof Harshitha'], 'hours_per_week': 2},
    ],
    'VIII': [
        {'code': 'BAD801', 'type': 'Theory', 'faculty': 'Prof Rohith', 'hours_per_week': 3},
        {'code': 'BDS802', 'type': 'Theory', 'faculty': 'Prof SV Prasad', 'hours_per_week': 3},
        {'code': 'BXX813x', 'type': 'Theory', 'faculty': 'Prof Sapna', 'hours_per_week': 3},
        {'code': 'BXX854x', 'type': 'Theory', 'faculty': 'Prof Sashi', 'hours_per_week': 3},
        {'code': 'BIKS809', 'type': 'Theory', 'faculty': 'Prof Harshitha', 'hours_per_week': 3},
        {'code': 'BCD885', 'type': 'Lab', 'faculty': ['Prof Rohith','Prof Chaithanya'], 'hours_per_week': 4},
        {'code': 'BCSL806', 'type': 'Lab', 'faculty': ['Prof SV Prasad', 'Prof Nagraj'], 'hours_per_week': 2},
        {'code': 'BXX857x', 'type': 'Lab', 'faculty': ['Prof Akhila', 'Prof Harshitha'], 'hours_per_week': 2},
    ],
}

# Helper function to generate empty timetable
def generate_empty_timetable():
    return [[None for _ in range(SLOTS_PER_DAY)] for _ in range(len(DAYS))]

# Genetic Algorithm
class TimetableGA:
    def __init__(self, courses_by_semester, population_size=30, generations=100):
        self.courses_by_semester = courses_by_semester
        self.population_size = population_size
        self.generations = generations
        self.population = []

    def initialize_population(self):
        self.population = []
        for _ in range(self.population_size):
            chromosome = {}
            for sem, courses in self.courses_by_semester.items():
                timetable = generate_empty_timetable()
                slots = list((day, slot) for day in range(len(DAYS)) for slot in range(SLOTS_PER_DAY))
                random.shuffle(slots)

                for course in courses:
                    hrs = course['hours_per_week']
                    placed = 0
                    if course['type'] == 'Lab':
                        while placed < hrs:
                            for d in range(len(DAYS)):
                                for s in range(SLOTS_PER_DAY - 1):
                                    if timetable[d][s] is None and timetable[d][s+1] is None and (s, s+1) not in BREAKS + LUNCH:
                                        timetable[d][s] = course['code']
                                        timetable[d][s+1] = course['code']
                                        placed += 2
                                        break
                                if placed >= hrs:
                                    break
                    else:
                        for d, s in slots:
                            if timetable[d][s] is None and (s,) not in BREAKS + LUNCH:
                                timetable[d][s] = course['code']
                                placed += 1
                                if placed == hrs:
                                    break
                chromosome[sem] = timetable
            self.population.append(chromosome)

    def fitness(self, chromosome):
        score = 0
        faculty_schedule = defaultdict(list)

        for sem, timetable in chromosome.items():
            courses = {c['code']: c for c in self.courses_by_semester[sem]}
            for d in range(len(DAYS)):
                for s in range(SLOTS_PER_DAY):
                    subject = timetable[d][s]
                    if subject is not None and subject in courses:
                        course = courses[subject]
                        if course['type'] == 'Lab' and s < SLOTS_PER_DAY - 1:
                            if timetable[d][s+1] != subject:
                                score -= 1  # Lab not continuous

                        # Priority handling
                        if course.get('priority'):
                            if course.get('preferred_day') and DAYS[d] == course['preferred_day']:
                                score += 2
                            if course.get('preferred_slot') == s:
                                score += 2

                        # Track faculty schedule
                        if isinstance(course['faculty'], list):
                            for f in course['faculty']:
                                faculty_schedule[f].append((d, s))
                        else:
                            faculty_schedule[course['faculty']].append((d, s))

        # Check for faculty conflicts
        for faculty, slots in faculty_schedule.items():
            seen = set()
            for slot in slots:
                if slot in seen:
                    score -= 1  # Faculty conflict
                seen.add(slot)
        return score

    def crossover(self, parent1, parent2):
        crossover_point = random.randint(0, len(self.courses_by_semester)-1)
        semesters = list(self.courses_by_semester.keys())
        child = {}
        for i, sem in enumerate(semesters):
            child[sem] = copy.deepcopy(parent1[sem] if i <= crossover_point else parent2[sem])
        return child

    def mutate(self, chromosome):
        # Randomly swap a couple of entries in one timetable
        sem = random.choice(list(chromosome.keys()))
        timetable = chromosome[sem]
        d1, d2 = random.randint(0, 5), random.randint(0, 5)
        s1, s2 = random.randint(0, 5), random.randint(0, 5)
        timetable[d1][s1], timetable[d2][s2] = timetable[d2][s2], timetable[d1][s1]

    def run(self):
        self.initialize_population()
        for _ in range(self.generations):
            self.population.sort(key=self.fitness, reverse=True)
            next_gen = self.population[:5]  # Elitism
            while len(next_gen) < self.population_size:
                parent1, parent2 = random.sample(self.population[:10], 2)
                child = self.crossover(parent1, parent2)
                if random.random() < 0.2:
                    self.mutate(child)
                next_gen.append(child)
            self.population = next_gen
        return self.population[0]  # Best timetable

# Run GA
ga = TimetableGA(courses_per_semester)
best_timetable = ga.run()

# Display
for sem, table in best_timetable.items():
    df = pd.DataFrame(table, index=DAYS, columns=[f"Slot {i+1}" for i in range(SLOTS_PER_DAY)])
    print(f"\n\n=== Timetable for {sem} Semester ===")
    print(df)
    
